<a href="https://colab.research.google.com/github/prakrutpatel/Context-RCNN-Tortoises/blob/main/Context%20RCNN%20Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install dependencies

In [ ]:
!pip install tensorflow_gpu==1.15

In [ ]:
!pip install numpy==1.19.5
!pip uninstall -y pycocotools
!pip install pycocotools --no-binary pycocotools

In [ ]:
%load_ext tensorboard

## Download and install TensorFlow framework

In [ ]:
%cd /content
!git clone --quiet https://github.com/tensorflow/models.git

!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk

!pip install -q Cython contextlib2 pillow lxml matplotlib pycocotools tf_slim

%cd /content/models/research
!protoc object_detection/protos/*.proto --python_out=.

import os
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/'

!python object_detection/builders/model_builder_test.py

#### Optionally mount google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Download base model

#### For this example we use Context R-CNN Resnet101 Model trained on Serengeti dataset
#### Follow intructions from Faster R-CNN training example

In [ ]:
%cd /content/
!wget "http://download.tensorflow.org/models/object_detection/context_rcnn_resnet101_snapshot_serengeti_2020_06_10.tar.gz"
!tar -xvf "context_rcnn_resnet101_snapshot_serengeti_2020_06_10.tar.gz"

In [ ]:
pipeline_fname = "/content/context_rcnn_resnet101_snapshot_serengeti_2020_06_10/pipeline.config"

In [ ]:
fine_tune_checkpoint = "/content/context_rcnn_resnet101_snapshot_serengeti_2020_06_10/model.ckpt"

## Configuring a Training Pipeline

#### Follow intructions from Faster R-CNN training example

In [ ]:
import os
pipeline_fname = os.path.join('/content/models/research/object_detection/samples/configs/', pipeline_file)

assert os.path.isfile(pipeline_fname), '`{}` not exist'.format(pipeline_fname)

In [ ]:
%cd /content/models/research/

#### Attention - Make changes to the config file to work with your dataset (number of classes, learning rates, image aspect ratios etc).


Take a look at [Context RCNN Config](https://github.com/prakrutpatel/Context-RCNN-Tortoises/blob/main/Examples/Context%20RCNN.config) for reference

Most importantly change these fields to match your example/environment:
1. num_classes
2. batch_size
3. fine_tune_checkpoint - change to fine_tune_checkpoint variable value
4. num_steps
5. label_map_path - you can use .pbtxt from faster rcnn training example
6. tf_record_input_reader
    - input_path: Location of the training tfrecord file made from Embedding+Context Maker file. Training Tfrecord with embedding and context
7. eval_input_reader
    - tf_record_input_reader
        - input_path: Location of the testing tfrecord file made from Embedding+Context Maker file. Testing Tfrecord with embedding and context

Take a look at the reference example config file above. It will give you some insight as to which parameters needs to be changed.
Once you get a better understanding of the training, you can modify the learning rates and steps config for optimal training.

In [ ]:
!cat {pipeline_fname}

In [ ]:
model_dir = 'training/'
# Optionally remove content in output model directory to fresh start.
!rm -rf {model_dir}
os.makedirs(model_dir, exist_ok=True)

## Train the model

For some reason installing lvis may cause problems with tensorflow. If this happens rerun all the pip install lines at the start of this notebook (without restarting session).

Do not rerun this cell after doing that

In [ ]:
!pip install lvis

In [ ]:
#Change steps and eval steps to match config file
!python /content/models/research/object_detection/model_main.py \
    --pipeline_config_path="/content/models/research/object_detection/samples/configs/context_rcnn_resnet101_snapshot_serengeti.config" \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps=4000 \
    --num_eval_steps=50

In [ ]:
!ls {model_dir}

In [ ]:
%tensorboard --logdir "/content/models/research/training/"

## Exporting a Trained Inference Graph
Once your training job is complete, you need to extract the newly trained inference graph, which will be later used to perform the object detection. This can be done as follows:

In [ ]:
import re
import numpy as np

lst = os.listdir(model_dir)
lst = [l for l in lst if 'model.ckpt-' in l and '.meta' in l]
steps=np.array([int(re.findall('\d+', l)[0]) for l in lst])
last_model = lst[steps.argmax()].replace('.meta', '')

last_model_path = os.path.join(model_dir, last_model)
print(last_model_path)

#input_type image_tensor is the only one required as from now on only images will be passed to the model
!python /content/models/research/object_detection/export_inference_graph.py \
    --input_type image_tensor \
    --input_shape 1,-1,-1,3 \
    --pipeline_config_path "/content/models/research/object_detection/samples/configs/context_rcnn_resnet101_snapshot_serengeti.config" \
    --trained_checkpoint_prefix {last_model_path} \
    --output_directory "/content/Context_RCNN_SS1_trained" \
    --use_side_inputs True \
    --side_input_shapes 1,2000,2057/1 \
    --side_input_names context_features,valid_context_size \
    --side_input_types float,int


#### Zip the saved model directory and save the zip to permanent storage

In [ ]:
!zip -r "/content/Context_RCNN_SS_trained_1.zip" "/content/Context_RCNN_SS_trained"

In [ ]:
!cp -r "/content/Context_RCNN_SS1_trained" "/content/drive/MyDrive/experimental/Context_RCNN_SS1_trained"